In [ ]:
import json
import os

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import tqdm
from itables import init_notebook_mode

import src.graphics as graphics

init_notebook_mode(all_interactive=True)

In [ ]:
COVERAGE_CSV = os.path.abspath("./resources/megadetector_coverage/MegaDetector_0_2.csv")
REPORTS_FOLDER = os.path.abspath("./reports/1_megadetector_results")

# subset_option = "all"
subset_option = "train"
# subset_option = "validation"
# subset_option = "test"

In [ ]:
report = pd.read_csv(COVERAGE_CSV, sep=";")

model = os.path.basename(COVERAGE_CSV).replace(".csv", "")
threshold = model.replace("MegaDetector_", "").replace("_", ".")

LOGS_PATH = os.path.abspath(f"./logs/MegaDetector_{threshold}")
os.makedirs(LOGS_PATH, exist_ok=True)

In [ ]:
if subset_option != "all":
    report = report[report["subset"] == subset_option]

In [ ]:
real_positives = report["binary_label"].sum()
covered_positives = report[
    (report["binary_label"] == 1) & (report["detector_label"] == 1)
].shape[0]

percentage_covered_positives = (
    (covered_positives / real_positives) * 100 if real_positives > 0 else 0
)
print(f"Percentage of covered positives: {percentage_covered_positives:.2f}%")
print(f"There is {real_positives} real positives")
print(f"There is {covered_positives} covered positives")

In [ ]:
def create_confusion_matrix(conf_matrix, threshold, subset_option):
    fig = go.Figure(
        data=go.Heatmap(
            z=conf_matrix,
            x=["Animal", "Empty"],
            y=["Empty", "Animal"],
            text=conf_matrix,
            texttemplate="%{text}",
            colorscale="Blues",
            showscale=True,
        )
    )

    fig.update_layout(
        title=f"Confusion Matrix MegaDetector {threshold} ({subset_option})",
        xaxis_title="Predicted value",
        yaxis_title="Actual value",
        xaxis=dict(tickmode="array", tickvals=[0, 1], ticktext=["Animal", "Empty"]),
        yaxis=dict(tickmode="array", tickvals=[0, 1], ticktext=["Empty", "Animal"]),
        template="seaborn",
        width=600,
        height=600,
    )

    return fig

In [ ]:
TP = ((report["binary_label"] == 1) & (report["detector_label"] == 1)).sum()
FP = ((report["binary_label"] == 0) & (report["detector_label"] == 1)).sum()
TN = ((report["binary_label"] == 0) & (report["detector_label"] == 0)).sum()
FN = ((report["binary_label"] == 1) & (report["detector_label"] == 0)).sum()

conf_matrix = np.array([[FP, TN], [TP, FN]])

accuracy_predict = (TP + TN) / (TP + TN + FP + FN)
recall_predict = TP / (TP + FN)
specificity_predict = TN / (TN + FP)
precision_predict = TP / (TP + FP)
f1_score_predict = (
    2 * (precision_predict * recall_predict) / (precision_predict + recall_predict)
)

confusion_matrix_fig = create_confusion_matrix(conf_matrix, threshold, subset_option)
confusion_matrix_fig.write_image(
    os.path.join(REPORTS_FOLDER, f"confusion_matrix_{threshold}_{subset_option}.png")
)

predicted_results = pd.DataFrame(
    {
        "accuracy_predict": [accuracy_predict],
        "precision_predict": [precision_predict],
        "recall_predict": [recall_predict],
        "specificity_predict": [specificity_predict],
        "f1_score_predict": [f1_score_predict],
    }
)
predicted_results.to_csv(
    os.path.join(LOGS_PATH, f"predicted_results_{threshold}_{subset_option}.csv"),
    index=False,
    sep=";",
)


print(predicted_results)
print()
confusion_matrix_fig.show()

In [ ]:
conf_matrix = np.array([[0, (FP + TN)], [(TP + FN), 0]])

fig = go.Figure(
    data=go.Heatmap(
        z=conf_matrix,
        x=["Animal", "Empty"],
        y=["Empty", "Animal"],
        text=conf_matrix,
        texttemplate="%{text}",
        colorscale="Greens",
        showscale=True,
    )
)

fig.update_layout(
    title="Ideal Confusion Matrix",
    xaxis_title="Predicted value",
    yaxis_title="Actual value",
    xaxis=dict(tickmode="array", tickvals=[0, 1], ticktext=["Animal", "Empty"]),
    yaxis=dict(tickmode="array", tickvals=[0, 1], ticktext=["Empty", "Animal"]),
    template="seaborn",
    width=600,
    height=600,
)
fig.show()
fig.write_image(
    os.path.join(REPORTS_FOLDER, f"confusion_matrix_IDEAL_{subset_option}.png")
)